In [4]:
import os
import tarfile
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
import re

ModuleNotFoundError: No module named 'nltk'

In [ ]:
# Download NLTK data files (only for the first time)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NITIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NITIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\NITIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
data_path = "C:\\Users\\Nitin\\Downloads\\twenty+newsgroups\\20_newsgroups.tar.gz"
extracted_path = "C:\\Users\\Nitin\\Downloads\\twenty+newsgroups\\20_newsgroups"

if not os.path.exists(extracted_path):
    with tarfile.open(data_path, 'r:gz') as tar:
        tar.extractall(path=os.path.dirname(data_path))


In [ ]:
# Load the data from the specific category
category_path = os.path.join(extracted_path, 'misc.forsale')
documents = []

In [ ]:
for file_name in os.listdir(category_path):
    file_path = os.path.join(category_path, file_name)
    with open(file_path, 'r', encoding='latin1') as file:
        documents.append(file.read())

In [ ]:
# Preprocess the text data
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    text = text.lower()
    text = text.strip()
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stop words and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [ ]:
documents = [preprocess(doc) for doc in documents]

In [ ]:
# Convert the text data into numerical form using TF-IDF
vectorizer = TfidfVectorizer(max_df=0.9, min_df=2, stop_words='english')
X = vectorizer.fit_transform(documents)

In [ ]:
# Apply LDA for topic modeling
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(X)

LatentDirichletAllocation(random_state=42)

In [ ]:
# Display the topics found by LDA
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(lda, vectorizer.get_feature_names_out(), no_top_words)

Topic 0:
sale speaker kou douglas offer hiram best toyota movie college
Topic 1:
drive sale card disk offer software mb email meg modem
Topic 2:
jthbachudeledu jay aprmagnusacsohiostateedu hayes forsale distribution love number copy walter
Topic 3:
khohuscedu room sale house southern aludrauscedu los angeles ibm pom
Topic 4:
comic st copy new wolverine hulk cover price spiderman sale
Topic 5:
ticket cd bitzmcolumbiadsuedu car sale asking airline clemson sega new
Topic 6:
junk mailing list catalog etiquette fusi damico company roger improper
Topic 7:
georgia institute technology atlanta sale receiver gtcprismgatechedu gtaprismgatechedu hydragatechedu cellar
Topic 8:
game hotel package voucher oplingerracrdgecom saletrade brian uk stony jonathan
Topic 9:
sale game lens camera offer news condition car asking best


In [ ]:
# Apply K-means for document clustering
num_clusters = 10
km = KMeans(n_clusters=num_clusters, random_state=42)
km.fit(X)

KMeans(n_clusters=10, random_state=42)

In [ ]:
# Attach the cluster labels to the documents
df = pd.DataFrame({'Document': documents, 'Cluster': km.labels_})

In [ ]:
# Display the clustering results
print(df.head())

                                            Document  Cluster
0  cantaloupesrvcscmuedurochesterudelgatechhowlan...        4
1  cantaloupesrvcscmueducrabapplesrvcscmuedufsece...        0
2  cantaloupesrvcscmuedudasnewsharvardeduogicseem...        3
3  newsgroups cantaloupesrvcscmueducrabapplesrvcs...        4
4  cantaloupesrvcscmueducrabapplesrvcscmuedufsece...        4


In [ ]:
# Save the clustering results to a CSV file
df.to_csv('C:\\Users\\Nitin\\Downloads\\document_clusters.csv', index=False)